**Импорт библиотек**

In [381]:
from google.colab import files # Для работы с файлами 
import numpy as np # Для работы с данными 
import pandas as pd # Для работы с таблицами
import matplotlib.pyplot as plt # Для вывода графиков
import os # Для работы с файлами
%matplotlib inline

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation # Слои для сети
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами и преобразования их в последовательности
from tensorflow.keras.preprocessing.sequence import pad_sequences # Метод для работы с последовательностями
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder # Метод кодирования тестовых лейблов
from sklearn.model_selection import train_test_split # Для разделения выборки на тестовую и обучающую
from google.colab import drive # Для работы с Google Drive
import time # Импортируем библиотеку time
import zipfile
from sklearn.externals import joblib

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


# Загрузка баз данных

In [ ]:
# Данные Каталога продукции ГИСП в разбивке по предприятиям (по ОГРН) и кодам ОКПД 2 производимой предприятиями продукции
product_GISP = pd.read_csv('/content/drive/MyDrive/Result_5.csv', encoding='utf-8', delimiter=',')
product_GISP.head()

In [ ]:
product_GISP['id'].max()

2038307

In [ ]:
# Данные Меры поддержки на сайте ГИСП
support_measures = pd.read_csv('/content/drive/MyDrive/Данные Меры поддержки на сайте ГИСП.csv', encoding='cp1251', delimiter=';')
support_measures.head()

,"""ID""","""URL""","""SMALL_NAME""","""FULL_NAME""","""NUMBER_NPA""","""DATE_NPA""","""NPA_NAME""","""DESCRIPTION""","""PURPOSE""","""OBJECTIVE""","""TYPE_MERA""","""TYPE_FORMAT_SUPPORT""","""SROK_VOZVRATA""","""PROCENT_VOZVRATA""","""GUARANTE_PERIODE""","""GUARANTEE_COST""","""APPLIANCE_ID""","""OKVED2""","""COMPLEXITY""","""AMOUNT_OF_SUPPORT""","""REGULARITY_SELECT""","""PERIOD""","""DOGOVOR""","""GOS_PROGRAM""","""EVENT""","""DOP_INFO""","""IS_NOT_ACTIVE""","""PRICHINA_NOT_ACT""","""REQ_ZAYAVITEL""","""REQUEST_PROJECT""","""IS_SOFINANCE""","""DOLYA_ISOFINANCE""","""BUDGET_PROJECT""","""POKAZATEL_RESULT""","""TERRITORIAL_LEVEL""","""REGION_ID""","""RESPONS_STRUCTURE""","""ORG_ID"""
0,12447263,https://gisp.gov.ru/support-measures/list/1244...,Субсидия на возмещение части затрат субъектов ...,Субсидия на возмещение части затрат субъектов ...,№ 267-П,23.07.2018,Постановление Правительства РК от 23.07.2018 N...,При возмещении части затрат субъектов малого и...,NaN,Государственная поддержка субъектов малого и с...,Финансовая поддержка,NaN,NaN,NaN,NaN,NaN,Не определено / не указано,NaN,"Проблемы в работе предприятия, связанные с опе...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,На дату подачи документов для участия в конкур...,NaN,False,NaN,NaN,NaN,Региональный,Республика Карелия,РОИВ,NaN
1,12447262,https://gisp.gov.ru/support-measures/list/1244...,Субсидия на возмещение части затрат субъектов ...,Субсидия на возмещение части затрат субъектов ...,№ 267-П,23.07.2018,Постановление Правительства РК от 23.07.2018 N...,При возмещении части затрат субъектов малого и...,NaN,Государственная поддержка субъектов малого и с...,Финансовая поддержка,NaN,NaN,NaN,NaN,NaN,Производство социально значимых товаров,NaN,Высокие затраты на создание новых производств ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,На дату подачи документов для участия в конкур...,NaN,False,NaN,NaN,NaN,Региональный,Республика Карелия,РОИВ,NaN
2,12447261,https://gisp.gov.ru/support-measures/list/1244...,Субсидия на возмещение части затрат субъектов ...,Субсидия на возмещение части затрат субъектов ...,№ 267-П,23.07.2018,Постановление Правительства РК от 23.07.2018 N...,При возмещении части затрат субъектов малого и...,NaN,Государственная поддержка субъектов малого и с...,Финансовая поддержка,NaN,NaN,NaN,NaN,NaN,Не определено / не указано,NaN,Нехватка оборотных средств,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,На дату подачи документов для участия в конкур...,NaN,False,NaN,NaN,NaN,Региональный,Республика Карелия,РОИВ,NaN
3,12447260,https://gisp.gov.ru/support-measures/list/1244...,Субсидия на компенсацию части затрат субъектов...,Субсидия на компенсацию части затрат субъектов...,№ 267-П,23.07.2018,Постановление Правительства РК от 23.07.2018 N...,При компенсации части затрат субъектов малого ...,NaN,Государственная поддержка субъектов малого и с...,Финансовая поддержка,NaN,NaN,NaN,NaN,NaN,Не определено / не указано,NaN,Нехватка оборотных средств,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,На дату подачи документов для участия в конкур...,NaN,False,NaN,NaN,NaN,Региональный,Республика Карелия,РОИВ,NaN
4,12447259,https://gisp.gov.ru/support-measures/list/1244...,Субсидия на компенсацию части затрат субъектов...,Субсидия на компенсацию части затрат субъектов...,№ 267-П,23.07.2018,Постановление Правительства РК от 23.07.2018 N...,При компенсации части затрат субъектов малого ...,NaN,Государственная поддержка субъектов малого и с...,Финансовая поддержка,NaN,NaN,NaN,NaN,NaN,Не определено / не указано,NaN,"Кадровые проблемы, Проблемы в работе предприят...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,На дату подачи документов для участия в конкур...,NaN,False,NaN,NaN,NaN,Региональный,Республика Карелия,РОИВ,NaN


In [ ]:
# Описание структуры данных Меры поддержки на сайте ГИСП.csv
description_support_measures = pd.read_csv('/content/drive/MyDrive/Описание структуры данных Меры поддержки на сайте ГИСП.csv', encoding='cp1251', delimiter=';')
description_support_measures.head()

,field name,english description,russian description,format
0,ID,Record ID,Идентификатор записи,integer
1,URL,Record URL,URL записи,string
2,SMALL_NAME,Name for the support measure,Краткое наименование меры поддержки,string
3,FULL_NAME,Full name for support measure,Полное наименование меры поддержки,string
4,NUMBER_NPA,NPA number for support measure,Номер НПА меры поддержки,string


In [ ]:
# Меры поддержки Минпромторга с кодами КБК.xlsx - Лист1.csv
code_kbk = pd.read_csv('/content/drive/MyDrive/Меры поддержки Минпромторга с кодами КБК.xlsx - Лист1.csv', delimiter=',')
code_kbk.head(10)

,1. Субсидии,Unnamed: 1,Unnamed: 2
0,NaN,Подпрограмма 1. Развитие транспортного и специ...,NaN
1,NaN,Основное мероприятие 1.1 Развитие автомобилест...,NaN
2,Д20,Субсидии российским организациям автомобилестр...,020 04 12 16 1 01 68793 811
3,Д20,Субсидии организациям автомобилестроения на пе...,020 04 12 16 1 01 64660 811
4,Д20,Субсидии российским производителям колесных тр...,020 04 12 16 1 01 66730 811
5,Д20,Субсидии российским производителям колесных тр...,020 04 12 16 1 01 66760 811
6,Д20,Субсидии российским производителям грузовых ко...,02 .04 12 16 1 01 68770 812
7,Д20,Субсидии российским кредитным организациям на ...,020 04 12 16 1 01 67640 811
8,Д20,Субсидии российским производителям колесных тр...,020 06 05 16 1 01 66750 811
9,NaN,Субсидии российским организациям автомобилестр...,NaN


In [ ]:
# Распред_ОКВЭД2_и_ОКПД2_отрасли и департаменты.xlsx - Sheet1.csv
OKVED2_OKPD2 = pd.read_csv('/content/drive/MyDrive/Распред_ОКВЭД2_и_ОКПД2_отрасли и департаменты.xlsx - Sheet1.csv', delimiter=',')
OKVED2_OKPD2[174:]

,МИНИСТЕРСТВО ПРОМЫШЛЕННОСТИ И ТОРГОВЛИ,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
174,Отрасль промышленности,Код ОКВЭД 2,Наименование кода ОКВЭД 2,Код ОКПД 2,Наименование продукции по коду ОКПД2
175,Департамент станкостроения и инвестиционного м...,NaN,NaN,NaN,NaN
176,Нефтегазовое машиностроение,20.59.5,"Производство прочих химических продуктов, не в...",20.59.56.150,"Катализаторы, не включенные в другие группиров..."
177,Электротехническая промышленность,23.19.7,Производство электрических изоляторов из стекла,23.19.25,Изоляторы электрические стеклянные
178,Электротехническая промышленность,23.43,Производство керамических изоляторов и изолиру...,23.43,Изоляторы электрические и арматура изолирующая...
...,...,...,...,...,...
3781,Химическая промышленность (Промышленные биотех...,20.16,Производство пластмасс и синтетических смол в ...,20.16.59.310,Полимеры природные и полимеры модифицированные...
3782,Химическая промышленность (Промышленные биотех...,10.89.4,Производство пищевых ферментов (в части касающ...,20.14.64,"Ферменты и прочие органические соединения, не ..."
3783,Химическая промышленность (Промышленные биотех...,20.14,Производство прочих основных органических хими...,20.14.34.,Кислоты поликарбоновые ароматические и кислоты...
3784,Химическая промышленность (Промышленные биотех...,20.20,Производство пестицидов и прочих агрохимически...,20.20,Пестициды и агрохимические продукты прочие (в ...


In [ ]:
# База данных Федерального казначейства по полученным финансовым мерам поддержки (субсидиям)
!rm -R /content/folder
!unzip -q '/content/drive/My Drive/folder.zip' -d /content/folder

# Парсинг таблицы "Предприятия ГИСП +данные о нефинансовых мерах"

In [21]:
# Предприятия ГИСП +данные о нефинансовых мерах 719, 878.xlsx - сводная информация.csv
company = pd.read_csv('/content/Предприятия ГИСП +данные о нефинансовых мерах 719, 878.xlsx - сводная информация.csv', delimiter=',', error_bad_lines=True)
company.to_csv('file.csv', header=False)
company = pd.read_csv('/content/file.csv')
company = company.drop('0', 1)
company.head()

,Полное наименование организации,Email,"Электронные площадки торгов, на которых зарегистрировано и работает предприятие",ОКВЭД2,Тип предприятия,"Вид деятельности, основной ТАСС","Вид деятельности, дополнительный ТАСС",Организационно правовая форма,Тип компании,Статус компании,Дата постановки на учет в налоговом органе,"Код налогового органа, поставившего на учет",Фактический адрес,Атрибуты предприятия,Название,Вид организации,Отрасль,ОГРН,ИНН,КПП,Регион,Адрес,Контактный email,Адрес сайта,Контактный телефон
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АНГСТРЕМ""",NaN,NaN,Производство элементов электронной аппаратуры ...,NaN,Производство элементов электронной аппаратуры ...,"Деятельность вспомогательная прочая, связанная...",НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Активная [7026697],23.06.1993,7735.0,"124460, ГОРОД МОСКВА, ГОРОД ЗЕЛЕНОГРАД, ПЛОЩАД...","Предприятие, оказывающее существенное влияние ...","АО ""АНГСТРЕМ""",Промышленное предприятие [8406271],Радиоэлектронная промышленность [258],1027700140930,7.735011e+09,773501001,Москва [3505],"ГОРОД МОСКВА, ПЛОЩАДЬ ШОКИНА, ДОМ 2 СТРОЕНИЕ 3",malinovskiy@angstrem.ru,www.angstrem.ru,+7 (499) 720-84-44
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОНЦЕРН ВОЗДУШНО-КОСМИЧЕ...",NaN,NaN,Производство оружия и боеприпасов [27574],NaN,Производство оружия и боеприпасов [27575],Производство коммуникационного оборудования [2...,АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026719],Организация [7026691],Активная [7026697],16.01.1995,7731.0,"121471, ГОРОД МОСКВА, УЛИЦА ВЕРЕЙСКАЯ, 41",Системообразующее предприятие [8690579] / Орга...,"АО ""КОНЦЕРН ВКО ""АЛМАЗ - АНТЕЙ""",Промышленное предприятие [8406271],Радиоэлектронная промышленность [258],1027739001993,7.731084e+09,997450001,Москва [3505],"121471, ГОРОД МОСКВА, УЛИЦА ВЕРЕЙСКАЯ, 41","antey@almaz-antey.ru, s.shartilova@almaz-antey...",http://www.almaz-antey.ru,+7(495) 276-2605 / +7(903) 260-7262 / +7(495) ...
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""НАУЧНО-ПРОИЗВОДСТВЕННОЕ ...",NaN,NaN,Производство оружия и боеприпасов [27575],NaN,Производство оружия и боеприпасов [27575],Лесоводство и прочая лесохозяйственная деятель...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Активная [7026697],02.07.2012,7100.0,"Российская Федерация, 30004, г. Тула, ул.Щегло...",Организация ОПК [8780665] / Системообразующее ...,"АО ""НПО ""СПЛАВ"" ИМ. А.Н. ГАНИЧЕВА""",Промышленное предприятие [8406271],Промышленность боеприпасов и спецхимии [256],1127154020311,7.105516e+09,710501001,Тульская область [3535],"ОБЛАСТЬ ТУЛЬСКАЯ, УЛИЦА ЩЕГЛОВСКАЯ ЗАСЕКА, 33",oktyabrskaya.lv@splavtula.ru,http://splav.org/,+7 (487) 225-55-64
3,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,NaN,NaN,Производство красителей и пигментов [27354] / ...,NaN,Производство оружия и боеприпасов [27575],Производство красителей и пигментов [27354] / ...,ФЕДЕРАЛЬНЫЕ ГОСУДАРСТВЕННЫЕ УНИТАРНЫЕ ПРЕДПРИЯ...,Организация [7026691],Активная [7026697],27.04.1999,4703.0,"188679, ОБЛАСТЬ ЛЕНИНГРАДСКАЯ, РАЙОН ВСЕВОЛОЖС...",Одобрено Минпромторгом России [8690599] / Орга...,"ФГУП ""ЗАВОД ИМЕНИ МОРОЗОВА""",Промышленное предприятие [8406271],Промышленность боеприпасов и спецхимии [256],1034700561895,4.703010e+09,470350001,Ленинградская область [3500],"188679, ОБЛАСТЬ ЛЕНИНГРАДСКАЯ, РАЙОН ВСЕВОЛОЖС...",info@zavodm.su,http://www.zavodm.su,+7(813) 703-5103
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОНЦЕРН РАДИОСТРОЕНИЯ ""В...",NaN,NaN,Производство коммуникационного оборудования [2...,NaN,Научные исследования и разработки в области ес...,Производство коммуникационного оборудования [2...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Активная [7026697],21.07.2003,7730.0,"121170, ГОРОД МОСКВА, ПРОСПЕКТ КУТУЗОВСКИЙ, ДО...",Системообразующее предприятие [8690579] / Орга...,"АО ""КОНЦЕРН ""ВЕГА""",Промышленное предприятие [8406271],Радиоэлектронная промышленность [258],1037730020844,7.730170e+09,773001001,Москва [3505],"121170, ГОРОД МОСКВА, ПРОСПЕКТ КУТУЗОВСКИЙ, ДО...",ebunto@vega.su,www.vega.su,NaN


In [22]:
company.count()

Полное наименование организации                                                    10130
Email                                                                                243
Электронные площадки торгов, на которых зарегистрировано и работает предприятие      709
ОКВЭД2                                                                              4205
Тип предприятия                                                                      816
Вид деятельности, основной ТАСС                                                    10335
Вид деятельности, дополнительный ТАСС                                               9330
Организационно правовая форма                                                      10492
Тип компании                                                                       10496
Статус компании                                                                    10496
Дата постановки на учет в налоговом органе                                         10463
Код налогового органа

In [23]:
company_copy = company.drop(company.columns[[0, 1, 2, 4, 8, 9, 11, 12, 13, 14 ,15, 19, 21, 22, 23, 24]], axis=1) 
company.drop(company.columns[[0, 1, 2, 4, 8, 9, 11, 12, 13, 14 ,15, 17, 18, 19, 21, 22, 23, 24]], axis=1, inplace=True) 

In [ ]:
company.head()

,ОКВЭД2,"Вид деятельности, основной ТАСС","Вид деятельности, дополнительный ТАСС",Организационно правовая форма,Дата постановки на учет в налоговом органе,Отрасль,Регион
0,Производство элементов электронной аппаратуры ...,Производство элементов электронной аппаратуры ...,"Деятельность вспомогательная прочая, связанная...",НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],23.06.1993,Радиоэлектронная промышленность [258],Москва [3505]
1,Производство оружия и боеприпасов [27574],Производство оружия и боеприпасов [27575],Производство коммуникационного оборудования [2...,АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026719],16.01.1995,Радиоэлектронная промышленность [258],Москва [3505]
2,Производство оружия и боеприпасов [27575],Производство оружия и боеприпасов [27575],Лесоводство и прочая лесохозяйственная деятель...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],02.07.2012,Промышленность боеприпасов и спецхимии [256],Тульская область [3535]
3,Производство красителей и пигментов [27354] / ...,Производство оружия и боеприпасов [27575],Производство красителей и пигментов [27354] / ...,ФЕДЕРАЛЬНЫЕ ГОСУДАРСТВЕННЫЕ УНИТАРНЫЕ ПРЕДПРИЯ...,27.04.1999,Промышленность боеприпасов и спецхимии [256],Ленинградская область [3500]
4,Производство коммуникационного оборудования [2...,Научные исследования и разработки в области ес...,Производство коммуникационного оборудования [2...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],21.07.2003,Радиоэлектронная промышленность [258],Москва [3505]


In [24]:
company.count()

ОКВЭД2                                         4205
Вид деятельности, основной ТАСС               10335
Вид деятельности, дополнительный ТАСС          9330
Организационно правовая форма                 10492
Дата постановки на учет в налоговом органе    10463
Отрасль                                        9875
Регион                                         9949
dtype: int64

In [25]:
print(len(company['ОКВЭД2'].unique()))
print(len(company['Вид деятельности, основной ТАСС'].unique()))
print(len(company['Вид деятельности, дополнительный ТАСС'].unique()))
print(len(company['Организационно правовая форма'].unique()))
print(len(company['Дата постановки на учет в налоговом органе'].unique()))
print(len(company['Отрасль'].unique()))
print(len(company['Регион'].unique()))

1285
933
9194
32
5261
180
85


In [26]:
company = company.dropna(axis=0, how='any')
company_copy = company_copy.dropna(axis=0, how='any')

In [27]:
company.count()

ОКВЭД2                                        3301
Вид деятельности, основной ТАСС               3301
Вид деятельности, дополнительный ТАСС         3301
Организационно правовая форма                 3301
Дата постановки на учет в налоговом органе    3301
Отрасль                                       3301
Регион                                        3301
dtype: int64

In [ ]:
print(len(company['ОКВЭД2'].unique()))
print(len(company['Вид деятельности, основной ТАСС'].unique()))
print(len(company['Вид деятельности, дополнительный ТАСС'].unique()))
print(len(company['Организационно правовая форма'].unique()))
print(len(company['Дата постановки на учет в налоговом органе'].unique()))
print(len(company['Отрасль'].unique()))
print(len(company['Регион'].unique()))

1210
557
3285
12
2513
113
82


In [28]:
company_np = company.to_numpy()
company_copy = company_copy.to_numpy()

In [29]:
company_np.shape

(3301, 7)

In [ ]:
company_np

array([['Производство элементов электронной аппаратуры [27608]',
        'Производство элементов электронной аппаратуры [27608]',
        'Деятельность вспомогательная прочая, связанная с перевозками [28284] / Деятельность агентов по оптовой торговле универсальным ассортиментом товаров [28431] / Торговля розничная преимущественно пищевыми продуктами, включая напитки, и табачными изделиями в неспециализированных магазинах [28606] / Торговля розничная прочая в неспециализированных магазинах [28610] / Торговля розничная вне магазинов, палаток, рынков [28766] / Разработка компьютерного программного обеспечения [28840] / Деятельность консультативная и работы в области компьютерных технологий [28841] / Деятельность по созданию и использованию баз данных и информационных ресурсов [28857] / Консультирование по вопросам коммерческой деятельности и управления [28963] / Деятельность в области архитектуры, связанная со зданиями и сооружениями [28967] / Научные исследования и разработки в области е

**формируем BoW**

In [30]:
# переводим даты в годы
for i in range(company_np.shape[0]):
    company_np[i][4] = company_np[i][4][-4:]

In [31]:
# формируем данные для обучения токенайзера и перевода в BoW
columns_0 = []   # все данные из нулевой колонки
columns_1 = []
columns_2 = []
columns_3 = []
columns_4 = []
columns_5 = []
columns_6 = []
for i in range(company_np.shape[0]):
  for j in range(company_np.shape[1]):
    if j == 0:
      columns_0.append(company_np[i][j])
    elif j == 1:
      columns_1.append(company_np[i][j])
    elif j == 2:
      columns_2.append(company_np[i][j])   
    elif j == 3:
      columns_3.append(company_np[i][j])
    elif j == 4:
      columns_4.append(company_np[i][j])      
    elif j == 5:
      columns_5.append(company_np[i][j])
    elif j == 6:
      columns_6.append(company_np[i][j])   

In [493]:
maxWordsCount = 2500
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!#$%&()*+-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff\n ', lower=True, split=',', oov_token='unknown', char_level=False)
tokenizer.fit_on_texts(columns_0)
items = list(tokenizer.word_index.items())

In [33]:
print(items[:120]) # Посмотрим n самых часто встречающихся слов
print("Размер словаря", len(items)) # Длина словаря

[('unknown', 1), ('и', 2), ('производство', 3), ('в', 4), ('оборудования', 5), ('изделий', 6), ('прочих', 7), ('для', 8), ('из', 9), ('аппаратуры', 10), ('области', 11), ('машин', 12), ('обработка', 13), ('средств', 14), ('не', 15), ('другие', 16), ('приборов', 17), ('группировки', 18), ('деятельность', 19), ('технических', 20), ('исследования', 21), ('прочие', 22), ('торговля', 23), ('инструментов', 24), ('научные', 25), ('разработки', 26), ('материалов', 27), ('металлических', 28), ('наук', 29), ('раздел', 30), ('естественных', 31), ('ремонт', 32), ('оружия', 33), ('включенных', 34), ('медицинских', 35), ('боеприпасов', 36), ('одежды', 37), ('оптовая', 38), ('конструкций', 39), ('металлов', 40), ('29015', 41), ('или', 42), ('частей', 43), ('кроме', 44), ('аппаратов', 45), ('назначения', 46), ('27574', 47), ('по', 48), ('автотранспортных', 49), ('производства', 50), ('строительных', 51), ('текстильных', 52), ('измерения', 53), ('химических', 54), ('их', 55), ('на', 56), ('целях', 57),

In [507]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
trainWordIndexes = tokenizer.texts_to_sequences(columns_0)
print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", columns_0[1][:50])
print("Он же в виде последовательности индексов: ", trainWordIndexes[25][:20], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               Производство оружия и боеприпасов [27574]
Он же в виде последовательности индексов:  [22, 165, 171, 120, 172, 3, 264, 62, 265, 3, 33, 2, 36, 47, 112, 13, 100, 113] 



In [35]:
# Преобразовываем полученные выборки из последовательности индексов в матрицы нулей и единиц по принципу Bag of Words
xTrain01 = tokenizer.sequences_to_matrix(trainWordIndexes)
print(xTrain01.shape)    
print(xTrain01[0][:100])

(3301, 2500)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [36]:
xTrain = []
xTrain.append(xTrain01)

**делаем это для остальных колонок**

In [37]:
# функция, которая принимает текст, возвращает BoW
def text_to_BoW(text):
    tokenizer = Tokenizer(num_words=maxWordsCount, filters='!#$%&()*+-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff\n ', lower=True, split=',', oov_token='unknown', char_level=False)
    tokenizer.fit_on_texts(text)
    trainWordIndexes = tokenizer.texts_to_sequences(text)
    xTrain01 = tokenizer.sequences_to_matrix(trainWordIndexes)

    return xTrain01

In [38]:
xTrain.append(text_to_BoW(columns_1))
xTrain.append(text_to_BoW(columns_2))
xTrain.append(text_to_BoW(columns_3))
xTrain.append(text_to_BoW(columns_4))
xTrain.append(text_to_BoW(columns_5))
xTrain.append(text_to_BoW(columns_6))

In [39]:
print(xTrain[0].shape)
print(len(xTrain))

(3301, 2500)
7


In [40]:
# для подачи в нейронку преобразуем в нампи массив
xTrain = np.array(xTrain)

In [41]:
xTrain.shape

(7, 3301, 2500)

In [43]:
xTrain = xTrain.reshape(3301, 7*2500)

In [44]:
xTrain.shape

(3301, 17500)

**Формируем yTrain**

In [45]:
sup_878 = pd.read_csv('/content/Предприятия ГИСП +данные о нефинансовых мерах 719, 878.xlsx - ОКПД из реестра РЭП.csv', delimiter=',')
sup_878.head()

,ОГРН предприятия,"Код ОКПД2 продукции, включенной в реестр РЭП по ПП 878"
0,1020100706526,26.51.66.134
1,1020202364302,32.50.50.190
2,1020202554558,26.30
3,1020203220916,26.20.14
4,1021000522663,27.90.40.150


In [48]:
sup_719 = pd.read_csv('/content/Предприятия ГИСП +данные о нефинансовых мерах 719, 878.xlsx - ОКПД из реестра по ПП 719.csv', delimiter=',')
sup_719.head()

,ОГРН предприятия,"Код ОКПД2 продукции, включенной в реестр по ПП 719"
0,304370231500087,13.92.24
1,1021603067188,28.21.13
2,304370231500087,13.92.24
3,1021603067188,28.99.39.190
4,1021603067188,28.99.39.190


In [49]:
# ОГРН мер поддержки 
OGRN_878 = sup_878['ОГРН предприятия'].values
OGRN_719 = sup_719['ОГРН предприятия'].values

In [50]:
xTrain.shape[0]

3301

In [309]:
yTrain = np.zeros(xTrain.shape[0])

In [310]:
yTrain.shape

(3301,)

In [311]:
company_copy.shape

(3301, 9)

In [312]:
company_copy[0]

array(['Производство элементов электронной аппаратуры [27608]',
       'Производство элементов электронной аппаратуры [27608]',
       'Деятельность вспомогательная прочая, связанная с перевозками [28284] / Деятельность агентов по оптовой торговле универсальным ассортиментом товаров [28431] / Торговля розничная преимущественно пищевыми продуктами, включая напитки, и табачными изделиями в неспециализированных магазинах [28606] / Торговля розничная прочая в неспециализированных магазинах [28610] / Торговля розничная вне магазинов, палаток, рынков [28766] / Разработка компьютерного программного обеспечения [28840] / Деятельность консультативная и работы в области компьютерных технологий [28841] / Деятельность по созданию и использованию баз данных и информационных ресурсов [28857] / Консультирование по вопросам коммерческой деятельности и управления [28963] / Деятельность в области архитектуры, связанная со зданиями и сооружениями [28967] / Научные исследования и разработки в области есте

In [313]:
for i in OGRN_878:
  for j in range(company_copy.shape[0]):
    if i in company_copy[j][-3]:
      yTrain[j] = 878

In [314]:
OGRN_719 = [str(i) for i in OGRN_719]

In [315]:
np.unique(yTrain)

array([  0., 878.])

In [316]:
for i in OGRN_719:
  for j in range(company_copy.shape[0]):
    if i in company_copy[j][-3]:
      yTrain[j] = 719

In [317]:
np.unique(yTrain)

array([  0., 719., 878.])

In [318]:
# количество нулей в массиве yTrain
num_zeroes = 0
for i in yTrain:
    if int(i) == 0:
        num_zeroes += 1
print(num_zeroes)

2701


**Найдем yTrain для финансовых мер поддержки**

In [319]:
fin = pd.read_csv('/content/drive/MyDrive/subs/subs1.csv', dtype=str)

In [320]:
fin[4:16]

,ИНН,ОГРН,KBK
4,1646021952,1071674002180,02004121610166730811
5,1646022089,1071674002432,02006051610166750811
6,1650032058,NaN,02004121610167130810
7,1650032058,NaN,02004121616713810242
8,1650032058,NaN,02004121610568847810
9,1650032058,NaN,02004121610568774811
10,1650032058,NaN,02004121610568773811
11,1650032058,1021602013971,02006051610166750811
12,1650032058,1021602013971,02004121610166730811
13,1650032058,NaN,02004121610167130811


In [321]:
fin.count()

ИНН     241
ОГРН    149
KBK     241
dtype: int64

In [322]:
ogrn = fin['ОГРН'].values

In [323]:
KBK = fin['KBK'].values

In [324]:
INN = fin['ИНН'].values

In [325]:
print(KBK.shape)
print(INN.shape)

(241,)
(241,)


In [326]:
company_copy.shape

(3301, 9)

In [327]:
INN[:10]

array(['0264004103', '0901050261', '1435332934', '1646021952',
       '1646021952', '1646022089', '1650032058', '1650032058',
       '1650032058', '1650032058'], dtype=object)

In [328]:
company_copy[0]

array(['Производство элементов электронной аппаратуры [27608]',
       'Производство элементов электронной аппаратуры [27608]',
       'Деятельность вспомогательная прочая, связанная с перевозками [28284] / Деятельность агентов по оптовой торговле универсальным ассортиментом товаров [28431] / Торговля розничная преимущественно пищевыми продуктами, включая напитки, и табачными изделиями в неспециализированных магазинах [28606] / Торговля розничная прочая в неспециализированных магазинах [28610] / Торговля розничная вне магазинов, палаток, рынков [28766] / Разработка компьютерного программного обеспечения [28840] / Деятельность консультативная и работы в области компьютерных технологий [28841] / Деятельность по созданию и использованию баз данных и информационных ресурсов [28857] / Консультирование по вопросам коммерческой деятельности и управления [28963] / Деятельность в области архитектуры, связанная со зданиями и сооружениями [28967] / Научные исследования и разработки в области есте

In [329]:
company_copy.shape

(3301, 9)

In [330]:
for i in range(company_copy.shape[0]):
  company_copy[i][-2] = int(company_copy[i][-2])

In [331]:
INN[:20]

array(['0264004103', '0901050261', '1435332934', '1646021952',
       '1646021952', '1646022089', '1650032058', '1650032058',
       '1650032058', '1650032058', '1650032058', '1650032058',
       '1650032058', '1650032058', '1650032058', '1650130591',
       '1650130591', '1650161470', '1654019570', '1654019570'],
      dtype=object)

In [332]:
list(INN).index('1650032058')

6

In [333]:
group_by_data = fin.groupby('ИНН').groups
innList = []
KVKList = []
for key, value in group_by_data.items():
  innList.append(key)
  KVKList.append(value)

nkvk = []
for el in KVKList:
  tmp = []
  if isinstance(el,list):
    for j in el:
      tmp.extend(j)
  else:
    tmp.extend(el)
    
  nkvk.append(tmp)

nkvk

[[0],
 [1],
 [2],
 [3, 4],
 [5],
 [6, 7, 8, 9, 10, 11, 12, 13, 14],
 [15, 16],
 [17],
 [18, 19],
 [20],
 [21],
 [22],
 [23],
 [24, 25, 26, 27, 28],
 [29],
 [30],
 [31],
 [32, 33],
 [34],
 [35],
 [36],
 [37, 38, 39],
 [40],
 [41, 42],
 [43],
 [44, 45],
 [46],
 [47, 48, 49],
 [50, 51, 52],
 [53, 54],
 [55],
 [56, 57, 58],
 [59],
 [60, 61],
 [62, 63, 64, 65],
 [66, 67, 68, 69, 70],
 [71],
 [72],
 [73, 74],
 [75, 76, 77, 78, 79, 80, 81],
 [82],
 [83, 84, 85],
 [86],
 [87],
 [88, 89, 90, 91, 92],
 [93, 94, 95, 96, 97, 98],
 [99, 100],
 [101, 102, 103, 104],
 [105, 106],
 [107],
 [108],
 [109],
 [110],
 [111],
 [112],
 [113],
 [114],
 [115, 116, 117, 118],
 [119],
 [120],
 [121],
 [122, 123],
 [124],
 [125, 126],
 [127],
 [128, 129, 130, 131, 132, 133, 134],
 [135, 136],
 [137, 138],
 [139, 140, 141, 142, 143],
 [144, 145],
 [146, 147, 148, 149, 150, 151, 152, 153, 154, 155],
 [156, 157, 158, 159, 160],
 [161, 162, 163],
 [164, 165, 166, 167],
 [168],
 [169],
 [170, 171, 172, 173],
 [174],
 

In [334]:
INNCodes = list(dict.fromkeys(fin['ИНН'].values))
nkvkCodes = [fin['KBK'].values[i] for i in nkvk]

In [335]:
yTrain = [int(i) for i in yTrain]
yTrain = [str(i) for i in yTrain]
yTrain = list(yTrain)


In [336]:
for nr,i in enumerate(INNCodes):
  for j in range(company_copy.shape[0]):
    if i in str(int(company_copy[j][-2])):
      ind = list(INNCodes).index(i) # присваивает первый КВК в списке.
      yTrain[j] = nkvkCodes[ind]

**Сделаем это для остальных json файлов**

In [338]:
# для этого создадим функцию
def for_json(path):
    fin = pd.read_csv(path, dtype=str)
    KBK = fin['KBK'].values
    INN = fin['ИНН'].values

    group_by_data = fin.groupby('ИНН').groups
    innList = []
    KVKList = []
    for key, value in group_by_data.items():
      innList.append(key)
      KVKList.append(value)

    nkvk = []
    for el in KVKList:
      tmp = []
      if isinstance(el,list):
        for j in el:
          tmp.extend(j)
      else:
        tmp.extend(el)
        
      nkvk.append(tmp)

    INNCodes = list(dict.fromkeys(fin['ИНН'].values))
    nkvkCodes = [fin['KBK'].values[i] for i in nkvk]

    for nr,i in enumerate(INNCodes):
      for j in range(company_copy.shape[0]):
        if i in str(int(company_copy[j][-2])):
          ind = list(INNCodes).index(i) # присваивает первый КВК в списке.
          yTrain[j] = nkvkCodes[ind]
    return yTrain

In [339]:
#yTrain = for_json('/content/drive/MyDrive/subs/subs1.csv')

In [340]:
yTrain = for_json('/content/drive/MyDrive/subs/subs2.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs3.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs4.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs5.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs6.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs7.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs8.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs9.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs10.csv')

In [341]:
yTrain

['0',
 array(['0200412165П368853811'], dtype=object),
 array(['02004121656056810242'], dtype=object),
 '0',
 array(['02004124410566801811'], dtype=object),
 array(['020041216Ш6650452530'], dtype=object),
 array(['02004121926312451530'], dtype=object),
 '0',
 array(['02004121656467810242'], dtype=object),
 array(['0200412166D564046811', '02004121926268451530'], dtype=object),
 '0',
 array(['02004121610468760811'], dtype=object),
 array(['02004121610468760811', '02004121670160670811',
        '02004121610568774811'], dtype=object),
 '0',
 array(['02004121656056810242', '02004121656056810242'], dtype=object),
 array(['02004121656483810242'], dtype=object),
 array(['02002094410164830811', '02004121656467810242'], dtype=object),
 array(['02002094410164830811'], dtype=object),
 array(['02004121650168350811', '02004111930168580812'], dtype=object),
 '0',
 '0',
 array(['02004121610564250811'], dtype=object),
 array(['02002094410264210810', '02004121710164740811',
        '02004121610568772811'

In [342]:
yTrain = for_json('/content/drive/MyDrive/subs/subs11.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs12.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs13.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs14.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs15.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs16.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs17.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs18.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs19.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs20.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs21.csv')
yTrain = for_json('/content/drive/MyDrive/subs/subs22.csv')

In [343]:
yTrain1 = yTrain.copy()

In [344]:
for i,el in enumerate(yTrain1):
  if isinstance(el,(list, tuple, set, np.ndarray)):
    #yTrain1[i] = yTrain1[i].tolist()
    yTrain1[i] = set(yTrain1[i])
    yTrain1[i] = list(yTrain1[i])


In [345]:
yTrain1

['0',
 ['02004121670154770540', '02004121610167640811', '02004112010490059611'],
 ['02004121656056810242'],
 '0',
 ['02004124410566801811'],
 ['020041216Ш6650452530'],
 ['02004121926312451530'],
 ['02004121926336451530'],
 ['02004121656467810242'],
 ['02004121926268451530', '0200412166D564046811'],
 '0',
 ['02004121610468760811'],
 ['02004121670160670811', '02004121610468760811', '02004121610568774811'],
 '0',
 ['02004121650168350811', '02004111930168580812', '02004121610468760811'],
 ['02004121656483810242'],
 ['0200412166D564046811'],
 ['8100412572I855270633'],
 ['02004121650168350811', '02004111930168580812'],
 ['02004121616470810242'],
 '0',
 ['02004121610564250811'],
 ['0200412165П360340811'],
 '0',
 ['02004121656483810242'],
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 ['0200412165П360340811'],
 ['02004121610167653811'],
 '0',
 '0',
 '0',
 '0',
 '719',
 ['02004121620560981811', '02004121646464810242'],
 '0',
 '0',
 ['02004112010468620811', '02004112010460730811'],
 ['02004121650168350811

In [346]:
lstWkvk=[]
for el in yTrain:
  if isinstance(el,(list, tuple, np.ndarray)):
    lstWkvk.extend(el)

lstWkvk.extend(['0','719','878'])
uniqueKVK = list(dict.fromkeys(lstWkvk)) #меры поддержки с их порядковыми номерами
len(uniqueKVK) 

191

In [347]:
uniqueKVK.index('0')
#elements

188

In [348]:
for i,el in enumerate(yTrain1):
  if isinstance(el,(list, tuple, np.ndarray)):
    for j,elements in enumerate(el):
      
      if elements in uniqueKVK:
        
        yTrain1[i][j] = uniqueKVK.index(elements)
  elif el in uniqueKVK:
    print(el)
    yTrain1[i] = uniqueKVK.index(el)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
719
719
0
0
0
0
0
0
0
0
719
0
719
0
0
0
0
0
0
0
0
0
719
0
0
0
0
719
0
0
719
719
0
0
0
0
0
0
0
0
0
0
719
0
0
719
0
719
719
0
0
0
0
0
0
0
0
0
0
719
0
0
0
719
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
719
0
0
0
719
719
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
719
0
0
719
0
0
0
719
0
719
0
0
719
0
0
0
719
0
0
0
0
0
0
719
719
719
719
0
0
0
719
0
719
0
0
719
719
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
719
719
0
0
0
0
0
0
0
719
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
719
0
0
0
0
719
0
0
0
0
0
719
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
719
719
719
0
719
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
719
0
0
0
0
0
719
719
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
719
0
0
0
0
0
0
0
0
0


In [349]:
to_categorical([0,1,2], 10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [350]:
for i,el in enumerate(yTrain1):
  yTrain1[i] = to_categorical(yTrain1[i], len(uniqueKVK))

In [361]:
a = np.array([[0,1,2],[0,0,1]])

a.sum(axis = 0)

array([0, 1, 3])

In [355]:
y = yTrain1.copy()

In [367]:
np.array(yTrain1).shape

(3301, 191)

In [363]:
for i,el in enumerate(yTrain1):
  if len(el)<190:
    yTrain1[i] = el.sum(axis = 0)

In [368]:
yTr = yTrain1.copy()

In [384]:
yTrain = np.array(yTr)
yTrain.shape

(3301, 191)

In [372]:
#np.save('/content/xTrain_for_hakaton', xTrain)
#np.save('/content/yTrain_for_hakaton', yTrain)

In [388]:
xTrain = np.load('/content/xTrain_for_hakaton.npy')
yTrain = np.load('/content/yTrain_for_hakaton.npy')

In [389]:
xTrain, xTest, yTrain, yTest = train_test_split(xTrain, yTrain, test_size = 0.15, )

In [416]:
print(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)

(2805, 17500) (496, 17500) (2805, 191) (496, 191)


In [562]:
def getXtest(descriptions):
  descriptions[4] = descriptions[4][-4:]
  out = []
  lst = [0,1,2,3,4,5,8]
  for i in lst:
    xTr = text_to_BoWCust(descriptions[i])
    
    out.append(xTr)

  out = np.array(out)
  
  out = out.reshape(1,17500)
  return out
# функция, которая принимает текст, возвращает BoW

def text_to_BoWCust(text):

    trainWordIndexes = tokenizer.texts_to_sequences(text)
    
    
    xTrain01 = tokenizer.sequences_to_matrix(trainWordIndexes)
    out = xTrain01.sum(axis = 0)
    for num in out:
      out[out > 1] = 1
    

    return out

In [558]:
# словарь для мер поддержки
subsDic = pd.read_json('/content/subs_dump_1.json', lines = True)
subsDic.head()

kbkCode = []
purpose = []
for i in range(len(subsDic)):
  try:
    kbkCode.append(subsDic['plansNormalized'][i][0]['kbkCode'])
    purpose.append(subsDic['plansNormalized'][i][0]['purpose'])
    if len(subsDic['plansNormalized'][i][0]['kbkCode'])<5:
      print(i)
  except:
    continue

print(len(kbkCode))
print(len(purpose))

myDict = dict(zip(kbkCode, purpose)
##

5174
5174


In [588]:
myDict1 = joblib.load('/content/myDict.pkl') # Второй словарь - получше

# Нейронная сеть

In [422]:
model = Sequential()

model.add(BatchNormalization())
model.add(Dense(2000, activation='linear'))
model.add(Dense(1000, activation = 'linear'))
model.add(Dense(1000, activation = 'linear'))

model.add(Dense(191, activation='sigmoid'))

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = Adam(0.00001))

In [423]:
model.fit(xTrain,yTrain, batch_size=64, epochs=5, validation_data=(xTest, yTest))

Epoch 1/5
44/44 [==============================] - 22s 466ms/step - loss: 5.7111 - accuracy: 0.2895 - val_loss: 2.8130 - val_accuracy: 0.6915
Epoch 2/5
44/44 [==============================] - 20s 455ms/step - loss: 2.1065 - accuracy: 0.7202 - val_loss: 3.2160 - val_accuracy: 0.6935
Epoch 3/5
44/44 [==============================] - 20s 455ms/step - loss: 1.3141 - accuracy: 0.7940 - val_loss: 3.7900 - val_accuracy: 0.6935
Epoch 4/5
44/44 [==============================] - 20s 459ms/step - loss: 1.3668 - accuracy: 0.8309 - val_loss: 4.4144 - val_accuracy: 0.6935
Epoch 5/5
44/44 [==============================] - 20s 459ms/step - loss: 1.6634 - accuracy: 0.7856 - val_loss: 5.3246 - val_accuracy: 0.6935


In [438]:
ans = []
for i in range(10):
  result = model.predict(np.expand_dims(xTest[i], axis = 0))
  print(i, np.where(result>0.97)[1])
  ans.append(np.where(result>0.97)[1])

0 [ 11  14  20  23  27  37  40  43  46  58  67  73  75 116 188 189]
1 [ 11  14  18  20  23  27  37  40  43  46  58  67  73  75 188 189]
2 [ 11  14  19  20  23  27  37  40  43  46  57  58  67  73  75 116 188 189]
3 [ 11  14  19  20  23  27  37  40  46  67  75 116 188 189]
4 [ 11  14  18  19  20  23  25  26  27  37  40  43  46  50  67  72  73  75
  89 188 189]
5 [ 11  14  19  20  23  27  37  40  46  67  75 116 188 189]
6 [ 18  19  20  25  26  27  37  40  41  46  67  75 145 149 188 189 190]
7 [ 11  14  19  20  23  27  37  40  46  67  75 188 189]
8 [ 11  14  20  23  37  40  43  46  58  67  73  75 188 189]
9 [ 11  14  19  20  23  27  37  40  46  57  58  67  75 116 188 189]


In [443]:
# для нулевого элемента в xTest:
for el in ans[0]:
  print('Идентификационный номер поддержки -', uniqueKVK[el])

Идентификационный номер поддержки - 02004121610468760811
Идентификационный номер поддержки - 02004121650168350811
Идентификационный номер поддержки - 0200412165П360340811
Идентификационный номер поддержки - 02004121646464810242
Идентификационный номер поддержки - 020041216Г6469810242
Идентификационный номер поддержки - 02004122020368600811
Идентификационный номер поддержки - 02004121620560980811
Идентификационный номер поддержки - 0200412165П360341811
Идентификационный номер поддержки - 0200412165П368851811
Идентификационный номер поддержки - 02004121630264620811
Идентификационный номер поддержки - 02004121610368775811
Идентификационный номер поддержки - 02004121660168460812
Идентификационный номер поддержки - 02004121610368779811
Идентификационный номер поддержки - 0200412165П368853811
Идентификационный номер поддержки - 0
Идентификационный номер поддержки - 719


# Ф-ция для обработки данных предприятия перед подачей в НС

Вводим следующие данные:
- ОКВЭД, 
- Вид деятельности, основной ТАСС, 
- Вид деятельности, дополнительный ТАСС, 
- Организационно Правовая форма, 
- Дата постановки на учет в налоговом органе, 
- Отрасль, 
- Регион

*Все эти данные можно получить из ЕГРЮЛ по ИНН или ОГРН

In [582]:
# пример данных, подаваемых в НС
for el in company_copy[45]:
  print(el)
  print('***')

Производство инструментов и приборов для измерения, тестирования и навигации [27647] / Научные исследования и разработки в области естественных и технических наук прочие [29015]
***
Научные исследования и разработки в области естественных и технических наук прочие [29015]
***
Производство приборов и аппаратуры для измерения электрических величин или ионизирующих излучений [27651] / Производство приборов для контроля прочих физических величин [27652] / Производство приборов и аппаратуры для автоматического регулирования или управления [27654] / Производство аппаратов, применяемых в медицинских целях, основанных на использовании рентгеновского, альфа-, бета- и гамма-излучений [27661] / Ремонт электронного и оптического оборудования [27959] / Деятельность гостиниц и прочих мест для временного проживания [28306] / Деятельность предприятий общественного питания по прочим видам организации питания [28325] / Общая врачебная практика [29233] / Деятельность в области медицины прочая, не включен

In [583]:
# предобработка данных в удобном для НС формате
dataTest = getXtest(company_copy[45])

In [584]:
result = model.predict(dataTest) # результат работы НС

In [585]:
# обработка данных НС
ans = []

ans.append(np.where(result>0.97)[1])
# для нулевого элемента в xTest:
print('рекомендуемые меры поддержки')
print('_____________________________')
for el in ans[0]:
  print('Идентификационный номер поддержки -', uniqueKVK[el])

рекомендуемые меры поддержки
_____________________________
Идентификационный номер поддержки - 02004121610468760811
Идентификационный номер поддержки - 02004121650168350811
Идентификационный номер поддержки - 0200412165П360340811
Идентификационный номер поддержки - 02004121646464810242
Идентификационный номер поддержки - 020041216Г6469810242
Идентификационный номер поддержки - 02004122020368600811
Идентификационный номер поддержки - 02004121620560980811
Идентификационный номер поддержки - 0200412165П368851811
Идентификационный номер поддержки - 02004121630264620811
Идентификационный номер поддержки - 02004121610368775811
Идентификационный номер поддержки - 02004121610368779811
Идентификационный номер поддержки - 0200412165П368853811
Идентификационный номер поддержки - 0
Идентификационный номер поддержки - 719


In [589]:
myDict1['02004121630264620811']

'Субсидия в целях возмещения затрат Получателя, связанных с поддержкой производства и реализации изделий народных художественных промыслов'